<a href="https://colab.research.google.com/github/rajbeniwal710/Spam_Collection_Model/blob/main/spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Spam Classification Model

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Untitled/SMSSpamCollection (1).tsv', sep = '\t')
data.columns = ['label', 'text']

In [8]:
#loading libraries and modules
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:

wn = nltk.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')

def count_punct(text):
  count = sum([1 for char in text if char in string.punctuation])
  return round(count/(len(text) - text.count(' ')),3)*100

data['% punct'] = data['text'].apply(lambda x: count_punct(x))
data['text_len'] = data['text'].apply(lambda x: len(x) - x.count(' '))

def clean_text(text):
  text1 = "".join([char.lower() for char in text if char not in string.punctuation])
  tokens = re.split('\W+', text1)
  text_stem = [wn.lemmatize(word) for word in tokens if word not in stopwords]
  return text_stem

tfidf = TfidfVectorizer(analyzer = clean_text)
x = tfidf.fit_transform(data['text'])

In [10]:
x_features = pd.concat([data['text_len'], data['% punct'], pd.DataFrame(x.toarray())], axis = 1)
x_features.head()

,text_len,% punct,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,8871,8872,8873,8874,8875,8876,8877,8878,8879,8880,8881,8882,8883,8884,8885,8886,8887,8888,8889,8890,8891,8892,8893,8894,8895,8896,8897,8898,8899,8900,8901,8902,8903,8904,8905,8906,8907,8908,8909,8910
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Random Forest Classification

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
rf = RandomForestClassifier(n_jobs=-1)
K_fold = KFold(n_splits=5)
cross_val_score(rf, x_features, data['label'], cv=K_fold, scoring='accuracy',n_jobs=-1)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


array([0.97755835, 0.97935368, 0.97394429, 0.96585804, 0.97394429])

Grid Search Random Forest classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_features, data['label'], test_size = 0.2)

In [ ]:
def train_rf(n_est, depth):
  rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1).fit(x_train,y_train)
  y_pred = rf.predict(x_test)
  precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
  print('Est: {} / Depth: {} ------- Precision: {} / Recall: {}'.format(
      n_est, depth, round(precision,3), round(recall, 3)
  ))

In [ ]:
rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1).fit(x_train,y_train)

In [ ]:
for n_est in (10,50,100):
  for depth in (10,20,30,None):
    train_rf(n_est, depth)

Est: 10 / Depth: 10 ------- Precision: 1.0 / Recall: 0.229
Est: 10 / Depth: 20 ------- Precision: 1.0 / Recall: 0.503
Est: 10 / Depth: 30 ------- Precision: 0.981 / Recall: 0.669
Est: 10 / Depth: None ------- Precision: 0.978 / Recall: 0.834
Est: 50 / Depth: 10 ------- Precision: 1.0 / Recall: 0.318
Est: 50 / Depth: 20 ------- Precision: 1.0 / Recall: 0.618
Est: 50 / Depth: 30 ------- Precision: 0.983 / Recall: 0.745
Est: 50 / Depth: None ------- Precision: 0.993 / Recall: 0.86
Est: 100 / Depth: 10 ------- Precision: 1.0 / Recall: 0.204
Est: 100 / Depth: 20 ------- Precision: 1.0 / Recall: 0.58
Est: 100 / Depth: 30 ------- Precision: 1.0 / Recall: 0.732
Est: 100 / Depth: None ------- Precision: 1.0 / Recall: 0.854


In [ ]:
from sklearn.model_selection import  GridSearchCV

In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150,300],
         'max_depth': [30,60,90,None]}

gs = GridSearchCV(rf, param, cv = 5, n_jobs=-1)
gs_fit = gs.fit(x_features, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,23.552762,0.250486,0.233416,0.005969,None,150,"{'max_depth': None, 'n_estimators': 150}",0.980251,0.977558,0.976640,0.966757,0.969452,0.974132,0.005134,1
7,21.692076,0.477811,0.213619,0.007420,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979354,0.977558,0.972147,0.969452,0.971249,0.973952,0.003821,2
8,41.579406,0.446287,0.355537,0.004304,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.979354,0.976661,0.973944,0.966757,0.970350,0.973413,0.004468,3
3,1.508750,0.064016,0.085936,0.019276,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.975763,0.976661,0.972147,0.964061,0.975741,0.972875,0.004671,4
11,43.279598,5.924815,0.362605,0.043699,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.976661,0.973046,0.964960,0.970350,0.972515,0.004575,5


Gradient Descent

In [14]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_features, data['label'], test_size = 0.2)

In [10]:
def train_gb(est, max_depth, lr):
  gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr).fit(x_train, y_train)
  y_pred = gb.predict(x_test)
  precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
  print('Est: {} / Depth: {} / lr: {} ------- Precision: {} / Recall: {}'.format(
      n_est, depth,lr, round(precision,3), round(recall, 3)
  ))


In [ ]:
for n_est in (50,100,150):
  for depth in (3,7,11,15):
    for lr in (0.01,0.1,1):
      train_gb(n_est, depth,lr)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 3 / lr: 0.01 ------- Precision: 0.0 / Recall: 0.0
Est: 50 / Depth: 3 / lr: 0.1 ------- Precision: 0.93 / Recall: 0.745
Est: 50 / Depth: 3 / lr: 1 ------- Precision: 0.884 / Recall: 0.801
Est: 50 / Depth: 7 / lr: 0.01 ------- Precision: 1.0 / Recall: 0.019
Est: 50 / Depth: 7 / lr: 0.1 ------- Precision: 0.956 / Recall: 0.801
Est: 50 / Depth: 7 / lr: 1 ------- Precision: 0.934 / Recall: 0.789


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 11 / lr: 0.01 ------- Precision: 0.0 / Recall: 0.0
Est: 50 / Depth: 11 / lr: 0.1 ------- Precision: 0.938 / Recall: 0.839
Est: 50 / Depth: 11 / lr: 1 ------- Precision: 0.91 / Recall: 0.814


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 15 / lr: 0.01 ------- Precision: 0.0 / Recall: 0.0
Est: 50 / Depth: 15 / lr: 0.1 ------- Precision: 0.942 / Recall: 0.814
Est: 50 / Depth: 15 / lr: 1 ------- Precision: 0.877 / Recall: 0.845
Est: 100 / Depth: 3 / lr: 0.01 ------- Precision: 0.965 / Recall: 0.509
Est: 100 / Depth: 3 / lr: 0.1 ------- Precision: 0.941 / Recall: 0.789
Est: 100 / Depth: 3 / lr: 1 ------- Precision: 0.891 / Recall: 0.814
Est: 100 / Depth: 7 / lr: 0.01 ------- Precision: 0.955 / Recall: 0.658
Est: 100 / Depth: 7 / lr: 0.1 ------- Precision: 0.95 / Recall: 0.832
Est: 100 / Depth: 7 / lr: 1 ------- Precision: 0.942 / Recall: 0.801
Est: 100 / Depth: 11 / lr: 0.01 ------- Precision: 0.959 / Recall: 0.733
Est: 100 / Depth: 11 / lr: 0.1 ------- Precision: 0.951 / Recall: 0.839
Est: 100 / Depth: 11 / lr: 1 ------- Precision: 0.878 / Recall: 0.807
Est: 100 / Depth: 15 / lr: 0.01 ------- Precision: 0.939 / Recall: 0.77
Est: 100 / Depth: 15 / lr: 0.1 ------- Precision: 0.95 / Recall: 0.832
Est: 100 / 

In [13]:
from sklearn.model_selection import  GridSearchCV

In [15]:
gb = GradientBoostingClassifier()
param = {'n_estimators': [100, 150],
         'max_depth': [7,11,15],
         'learning_rate' : [0.1]
         }

gs = GridSearchCV(gb, param, cv = 5, n_jobs=-1)
cv_fit = gs.fit(x_features, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,381.553477,7.076690,0.147079,0.009905,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.968582,0.979354,0.970350,0.966757,0.970350,0.971079,0.004346,1
1,259.134003,4.406547,0.223425,0.099677,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.969479,0.979354,0.969452,0.965858,0.969452,0.970719,0.004537,2
0,171.494390,9.629000,0.234480,0.097968,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.969479,0.981149,0.965858,0.967655,0.967655,0.970359,0.005515,3
4,323.656388,9.350095,0.245536,0.082604,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.967684,0.976661,0.970350,0.964960,0.971249,0.970181,0.003915,4
5,478.042426,61.046993,0.270600,0.054543,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.967684,0.977558,0.970350,0.965858,0.969452,0.970181,0.003998,5
